[View in Colaboratory](https://colab.research.google.com/github/pmanlukas/colab/blob/master/openapi_classifier.ipynb)

# openAPI spec classifier
This notebook is used to create a first proof of concept on a classifier for openAPI files. The classifier will be based on a neural network based architecture and will be implemented in Tensorflow, SciPy and Keras.

In [3]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

gpg: keybox '/tmp/tmpooevn20e/pubring.gpg' created
gpg: /tmp/tmpooevn20e/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [5]:
# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

KeyboardInterrupt: ignored

In [6]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [0]:
# Create a folder to use for the project data
!mkdir -p /content/drive/openapi-data/

In [8]:
!ls /content/drive/openapi-data/

mnist.py  path_labels.csv  pickle  structure_labels.csv


In [9]:
!pip install -q keras
import keras

Using TensorFlow backend.


In [10]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
!apt-get -qq install -y libarchive-dev && pip install -q -U libarchive
import libarchive

In [0]:
import os
os.chdir("drive/openapi-data")

In [13]:
!ls

mnist.py  path_labels.csv  pickle  structure_labels.csv


In [0]:
import pickle

In [0]:
training_data = dict()

with open('pickle/specs.pkl', 'rb') as handle:
    training_data = pickle.load(handle)

In [0]:
keys = list(training_data.keys())

In [17]:
training_data[keys[0]]

{'basePath': '/forex-quotes',
 'host': '1forge.com',
 'info': {'contact': {'email': 'contact@1forge.com',
   'name': '1Forge',
   'url': 'http://1forge.com'},
  'description': 'Stock and Forex Data and Realtime Quotes',
  'title': '1Forge Finance APIs',
  'version': '0.0.1',
  'x-apisguru-categories': ['financial'],
  'x-logo': {'backgroundColor': '#24292e',
   'url': 'https://api.apis.guru/v2/cache/logo/http_1forge.com_logo.png'},
  'x-origin': [{'format': 'swagger',
    'url': 'http://1forge.com/openapi.json',
    'version': '2.0'}],
  'x-preferred': True,
  'x-providerName': '1forge.com'},
 'paths': {'/quotes': {'get': {'description': 'Get quotes',
    'externalDocs': {'description': 'Find out more',
     'url': 'http://1forge.com/forex-data-api'},
    'responses': {'200': {'description': 'A list of quotes'}},
    'summary': 'Get quotes for all symbols',
    'tags': ['forex', 'finance', 'quotes']}},
  '/symbols': {'get': {'description': 'Symbol List',
    'externalDocs': {'descripti

In [0]:
from collections import Counter
from datetime import datetime
 
import json
 
from keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Dropout, Activation
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
 
import numpy as np
import pandas as pd

In [19]:
!ls

mnist.py  path_labels.csv  pickle  structure_labels.csv


In [0]:
labelsP = pd.read_csv("path_labels.csv").values
labelsS  = pd.read_csv("structure_labels.csv").values

In [21]:
labelsS.shape



(1016, 2)

In [22]:
labelsS

array([['1forge.com', 1],
       ['6-dot-authentiqio.appspot.com', 1],
       ['adafruit.com', 2],
       ...,
       ['zoom.us', 4],
       ['zoomconnect.com', 3],
       ['zuora.com', 3]], dtype=object)

## Prepare Labels and Specs for the model
In this part of the notebook, we will create three list. One contains the specs and the other two the labels for paths and for specs. The lists are later used as input for the actual preprocessing and then later the model.

In [23]:
labelsPath = list()
labelsStruct = list()


for labs in labelsS:
  labelsStruct.append(labs[1])

for labp in labelsP:
  labelsPath.append(labp[1])
  
print(str(len(labelsPath)))
print(str(len(labelsStruct)))

#labelsStruct = labelsS["cluster"].tolist()

1016
1016


In [24]:
print(labelsPath)
print(labelsStruct)

[1, 2, 3, 4, 5, 6, 7, 4, 4, 8, 3, 6, 8, 3, 4, 8, 6, 2, 4, 4, 2, 8, 6, 3, 4, 4, 8, 4, 9, 4, 4, 3, 3, 3, 4, 4, 6, 4, 8, 3, 9, 4, 4, 3, 3, 4, 3, 3, 3, 10, 4, 3, 3, 3, 8, 3, 3, 3, 4, 6, 1, 8, 4, 8, 6, 4, 6, 3, 3, 2, 6, 2, 3, 6, 9, 9, 4, 5, 1, 4, 8, 3, 4, 3, 6, 3, 3, 5, 4, 8, 8, 8, 9, 9, 8, 1, 4, 8, 3, 6, 4, 3, 3, 2, 9, 6, 6, 3, 8, 2, 3, 4, 5, 1, 6, 3, 8, 8, 6, 4, 8, 8, 4, 3, 4, 6, 4, 6, 2, 8, 4, 3, 9, 1, 6, 6, 3, 3, 4, 2, 3, 2, 6, 6, 4, 7, 1, 7, 7, 1, 7, 7, 7, 7, 7, 5, 1, 7, 7, 7, 7, 7, 7, 1, 7, 7, 7, 7, 7, 7, 7, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 9, 2, 7, 7, 2, 7, 7, 7, 2, 3, 5, 1, 8, 3, 9, 2, 4, 9, 2, 8, 9, 4, 9, 9, 9, 8, 2, 9, 2, 9, 9, 1, 2, 9, 8, 9, 5, 2, 2, 7, 4, 2, 2, 9, 7, 5, 5, 9, 9, 5, 5, 9, 9, 7, 9, 8, 7, 2, 8, 2, 9, 9, 5, 9, 9, 9, 2, 5, 5, 2, 5, 7, 2, 8, 9, 5, 1, 1, 9, 5, 9, 9, 5, 7, 1, 1, 1, 1, 1, 9, 5, 1, 1, 1, 1, 1, 1, 1, 9, 1, 1, 1, 1, 1, 7, 5, 5, 9, 4, 8, 9, 4, 2, 2, 4, 4, 6, 1, 5, 1, 3, 9, 8, 5, 7, 7, 9, 9, 8, 9, 7, 9, 4, 3, 9, 3, 4, 3, 2, 8, 5, 4, 8, 6, 2, 9, 4, 4, 5, 5, 7, 9,

### Create training and test split of the labels

In [0]:
Y_Path_train = labelsPath[:808]
Y_Path_test = labelsPath[808:]

Y_Struct_train = labelsStruct[:808]
Y_Struct_test = labelsStruct[808:]

In [159]:
print(len(Y_Path_train))
print(len(Y_Path_test))
print(len(Y_Struct_train))
print(len(Y_Struct_test))

808
208
808
208


In [0]:
with open('pickle/Y_Path_train.pickle', 'wb') as handle:
    pickle.dump(Y_Path_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('pickle/Y_Path_test.pickle', 'wb') as handle:
    pickle.dump(Y_Path_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('pickle/Y_Struct_train.pickle', 'wb') as handle:
    pickle.dump(Y_Struct_train, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
with open('pickle/Y_Struct_test.pickle', 'wb') as handle:
    pickle.dump(Y_Struct_test, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
specs_json = list()

In [0]:
import json

In [33]:
for spec in training_data:
  strJson = json.dumps(training_data[spec])
  specs_json.append(strJson)

print(str(len(specs_json)))
  
  

1016


In [34]:
specs_json[1]

'{"swagger": "2.0", "schemes": ["https"], "host": "6-dot-authentiqio.appspot.com", "basePath": "/", "info": {"contact": {"email": "hello@authentiq.com", "name": "Authentiq team", "url": "http://authentiq.io/support"}, "description": "Strong authentication, without the passwords.", "license": {"name": "Apache 2.0", "url": "http://www.apache.org/licenses/LICENSE-2.0.html"}, "termsOfService": "http://authentiq.com/terms/", "title": "Authentiq", "version": "6", "x-apisguru-categories": ["security"], "x-logo": {"backgroundColor": "#F26641", "url": "https://api.apis.guru/v2/cache/logo/https_www.authentiq.com_theme_images_authentiq-logo-a-inverse.svg"}, "x-origin": [{"format": "swagger", "url": "https://raw.githubusercontent.com/AuthentiqID/authentiq-docs/master/docs/swagger/issuer.yaml", "version": "2.0"}], "x-preferred": true, "x-providerName": "6-dot-authentiqio.appspot.com"}, "parameters": {"AuthentiqID": {"description": "Authentiq ID to register", "in": "body", "name": "body", "required"

In [160]:
X_train = specs_json[:808]
X_test = specs_json[808:]

print(len(X_train))
print(len(X_test))

808
208


## Preprocessing
In this part of the notebook we are going to preprocess the data itself. The data will be tokenized, preprocessed and stored as Training and Test sets

In [0]:
from keras.preprocessing.text import Tokenizer

Once fit, the Tokenizer provides 4 attributes that you can use to query what has been learned about your documents:

*   word_counts: A dictionary of words and their counts.
*   word_docs: A dictionary of words and how many documents each appeared in.
*   word_index: A dictionary of words and their uniquely assigned integers.
*   document_count:An integer count of the total number of documents that were used to fit the Tokenizer.


[source](https://machinelearningmastery.com/prepare-text-data-deep-learning-keras/)


In [0]:
tokenizer = Tokenizer(num_words=10000,filters='!"#$%&()*+,-/:;<=>?@[\\]^_`{|}~\t\n')

tokenizer

tokenizer.fit_on_texts(specs_json)



x_train = tokenizer.texts_to_matrix(X_train)
x_test = tokenizer.texts_to_matrix(X_test)

--------------------------------

In [138]:
x_train

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [137]:
x_test

array([[0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [0]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder

encoder = LabelBinarizer()
encoder.fit(labelsStruct)
y_train = encoder.transform(Y_Struct_train)
y_test = encoder.transform(Y_Struct_test)

In [155]:
y_train

array([[1, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       ...,
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0]])

In [141]:
len(x_train[0])

10000

In [163]:
print('y_train shape:', y_train.shape)

y_train shape: (808, 6)


In [164]:
print('x_train shape:', x_train.shape)

x_train shape: (808, 10000)


In [0]:
batch_size = 32
epochs = 2

---------------------------------------------------------------

## Model Architecture
In this section of the notebook the actual deep learning model is designed and the architecture presented before the actual training of it will take place.

In [0]:
model = Sequential()
model.add(Embedding(10000, 128, input_length=10000))
model.add(Dropout(0.2))
model.add(Conv1D(64, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))

model.add(Dense(6, activation='softmax',name = 'softmax_layer'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [184]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 10000, 128)        1280000   
_________________________________________________________________
dropout_46 (Dropout)         (None, 10000, 128)        0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 9996, 64)          41024     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 2499, 64)          0         
_________________________________________________________________
dropout_47 (Dropout)         (None, 2499, 64)          0         
_________________________________________________________________
lstm_13 (LSTM)               (None, 128)               98816     
_________________________________________________________________
dropout_48 (Dropout)         (None, 128)               0         
__________

In [187]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=3,
                    verbose=1,
                    validation_split=0.2)

Train on 646 samples, validate on 162 samples
Epoch 1/3
646/646 [==============================] - 150s 233ms/step - loss: 1.5604 - acc: 0.4381 - val_loss: 3.1695 - val_acc: 0.0123
Epoch 2/3
646/646 [==============================] - 147s 227ms/step - loss: 1.4530 - acc: 0.4721 - val_loss: 3.4883 - val_acc: 0.0123
Epoch 3/3
646/646 [==============================] - 148s 230ms/step - loss: 1.4190 - acc: 0.4721 - val_loss: 2.9588 - val_acc: 0.0123


In [0]:
model2 = Sequential()
model2.add(Dense(512, input_shape=(10000,)))
model2.add(Activation('relu'))
model2.add(Dropout(0.2))
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dropout(0.2))
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dropout(0.2))
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dropout(0.2))
model2.add(Dense(512))
model2.add(Activation('relu'))
model2.add(Dense(6))
model2.add(Activation('softmax'))

model2.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [190]:
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_62 (Dense)             (None, 512)               5120512   
_________________________________________________________________
activation_59 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_63 (Dense)             (None, 512)               262656    
_________________________________________________________________
activation_60 (Activation)   (None, 512)               0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_64 (Dense)             (None, 512)               262656    
__________

In [191]:
history2 = model2.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=15,
                    verbose=1,
                    validation_split=0.2)

Train on 646 samples, validate on 162 samples
Epoch 1/15
646/646 [==============================] - 4s 6ms/step - loss: 0.8199 - acc: 0.6889 - val_loss: 4.2391 - val_acc: 0.0309
Epoch 2/15
646/646 [==============================] - 0s 620us/step - loss: 0.3885 - acc: 0.8762 - val_loss: 2.5786 - val_acc: 0.0802
Epoch 3/15
646/646 [==============================] - 0s 613us/step - loss: 0.2496 - acc: 0.9149 - val_loss: 1.7607 - val_acc: 0.0864
Epoch 4/15
646/646 [==============================] - 0s 591us/step - loss: 0.1625 - acc: 0.9412 - val_loss: 2.5060 - val_acc: 0.1049
Epoch 5/15
646/646 [==============================] - 0s 589us/step - loss: 0.0871 - acc: 0.9721 - val_loss: 0.6664 - val_acc: 0.8272
Epoch 6/15
646/646 [==============================] - 0s 615us/step - loss: 0.1412 - acc: 0.9659 - val_loss: 1.3406 - val_acc: 0.4506
Epoch 7/15
646/646 [==============================] - 0s 592us/step - loss: 0.0890 - acc: 0.9675 - val_loss: 5.2496 - val_acc: 0.0864
Epoch 8/15
646/646

646/646 [==============================] - 0s 586us/step - loss: 0.1192 - acc: 0.9783 - val_loss: 8.7521 - val_acc: 0.0617
Epoch 15/15
646/646 [==============================] - 0s 605us/step - loss: 0.1719 - acc: 0.9690 - val_loss: 0.6700 - val_acc: 0.8889


### first training

In [192]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

208/208 [==============================] - 10s 50ms/step
Test score: 2.1362342834472656
Test accuracy: 0.014423076923076924


In [197]:
score = model2.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test score:', score[0])
print('Test accuracy:', score[1])

208/208 [==============================] - 0s 189us/step
Test score: 3.211094856262207
Test accuracy: 0.34615384615384615


### Evaluation

In [0]:
text_labels = encoder.classes_ 

for i in range(10):
    prediction = model.predict(np.array([x_test[i]]))
    predicted_label = text_labels[np.argmax(prediction)]
    print(test_posts.iloc[i][:50], "...")
    print('Actual label:' + test_tags.iloc[i])
    print("Predicted label: " + predicted_label + "\n")

In [0]:
y_softmax = model.predict(x_test)

y_test_1d = []
y_pred_1d = []

for i in range(len(y_test)):
    probs = y_test[i]
    index_arr = np.nonzero(probs)
    one_hot_index = index_arr[0].item(0)
    y_test_1d.append(one_hot_index)

for i in range(0, len(y_softmax)):
    probs = y_softmax[i]
    predicted_index = np.argmax(probs)
    y_pred_1d.append(predicted_index)

In [0]:
cnf_matrix = confusion_matrix(y_test_1d, y_pred_1d)
plt.figure(figsize=(24,20))
plot_confusion_matrix(cnf_matrix, classes=text_labels, title="Confusion matrix")
plt.show()